# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.pivot_table(values='debt', index='children', aggfunc=({'count', 'sum', 'mean'})).sort_values('children')

,count,mean,sum
children,,,
0,14091.0,0.075438,1063.0
1,4808.0,0.092346,444.0
2,2052.0,0.094542,194.0
3,330.0,0.081818,27.0
4,41.0,0.097561,4.0
5,9.0,0.000000,0.0


**Вывод:**\
По соотношениям можно сказать, что бездетные реже имеют просрочки.\
Однако, уверенно сказать, что с увеличением количества детей увеличивается и количество просрочек - нельзя,\
так как выбоорка данных по семьям с детьми явно меньше.\
Хотя тенденция есть. Нужно больше данных.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.pivot_table(values='debt', index='family_status', aggfunc=({'count', 'sum', 'mean'})).sort_values('mean', ascending=False)

,count,mean,sum
family_status,,,
Не женат / не замужем,2796.0,0.097639,273.0
гражданский брак,4134.0,0.093130,385.0
женат / замужем,12261.0,0.075606,927.0
в разводе,1189.0,0.070648,84.0
вдовец / вдова,951.0,0.066246,63.0


**Вывод:**\
Можем проследить отчетливую тенденцию в сторону уменьшения количества невозвратов\
с изменением семейного положения.\
Но скорее всего эта корреляция напрямую связана с жизненным опытом. Так как можно\
предположить, что изменение статуса напрямую связано с возрастом, а также получением\
некоторого "жизненного опыта", а вместе с ним "осознанности".\
Удобная и рабочая метрика, которую можно использовать.\

**Для уточнения корректности выводов**, а также чтобы проверить "возрастную" гипотезу\
нужно связаться с инженерами, для уточнения: есть ли техническая возможность\
уточнить конкретнее семейный статус и возраст клиента в момент появления задолженности.

**Вывод:**\
Не состоящие в браке и состоящие в гражданском браке - имеют самый рисковый коэфицент невозврата.\
У женатых этот показатель лучше на 20%.\
Лучше показатели и у овдовевших и состоящих в разводе, но при этом выборка в количестве сильно меньше.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.pivot_table(values='debt', index='total_income_category', aggfunc=({'count', 'sum', 'mean'})).sort_values('total_income_category')

,count,mean,sum
total_income_category,,,
A,25.0,0.080000,2.0
B,5014.0,0.070602,354.0
C,15921.0,0.084982,1353.0
D,349.0,0.060172,21.0
E,22.0,0.090909,2.0


**Вывод:**\
Прослеживается отчетливая парабола: пик задолженностей приходится на\
медианную группу дохода; в то время, как по краям наименьшие значения.\
Нужны дополнительные проперки.

*пытался найти через функцию параболические индексы, но не сообразил\
как их можно "подвязать" к выводу. прошу разъяснений, по возможности\
спасибо!*

**Вывод:**\
К сожалению, в анализе можем использовать только 2 категории: В и С. Так как количество строк остальных групп меньньше.\
По двум группам видим четкую тенденцию к увеличению коэфицента невыплат с уменьшением уровня дохода.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.pivot_table(values='debt', index='purpose_category', aggfunc=({'count', 'sum', 'mean'})).sort_values('mean')

,count,mean,sum
purpose_category,,,
операции с недвижимостью,10751.0,0.072551,780.0
проведение свадьбы,2313.0,0.079118,183.0
получение образования,3988.0,0.092528,369.0
операции с автомобилем,4279.0,0.093480,400.0


**Вывод:**\
Выборка широкая. Коэфиценты говорят сами за себя:\
1. кредиты на недвижимость и свадьбу имеют наименьшую просрочку
2. кредиты на автомобиль и образование - самые нестабильные к выплате

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 
1. Ошибка сбора данных
2. Несоответствие формата данных (например, int/str)
3. Неполные данные анкет (например, email)
4. Удаление данных

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 
Медианные значения для пропусков количественных данных позволяют использовать Дату без нарушения\
центральной предельной теоремы. Что крайне положительно влияет на точность выводов.

### Шаг 4: общий вывод.

**Общий вывод:**\
Получив Дату с данными заемщиков, на этапе предобработки данных заполнил количественные переенные медианными значениями для каждой группы.\
Явных проблем в выгрузке данных выявлено не было.\
Добавил колонку с ранжированием по зарплате, а также целям кредита для дальнейшего анализа и работы с гипотезами.\

Проанализировав данные, выполнив расчеты можно сказать:

1. Влияет ли наличие/количество детей - **не удалось** из-за узкой выборки
2. Влияние наличие семейных отношений положително влияей на возврат кредита
3. Платежеспособность повышается с уровнем дохода. Однако, нужно больше данных, чтобы установить зависимость
4. Более надежными кредитными целями являются: операции с недвижимостью; проведение свадьбы. Получение образования и операции с автомобилем же менее надежные

Рекомендации:
Чтобы выявить зависимости в гипотезах связаных с количеством детей и уровнем дохода хотелось бы более широкую Дату.